#  ChatMessageHistory

In [9]:
from langchain.memory import ChatMessageHistory
from rich import print as pp

In [6]:
history = ChatMessageHistory()
# Add more messages to the history
history.add_user_message("How can I assist you today?")
history.add_ai_message("I would like to know about Langchain.")

pp(history)


InMemoryChatMessageHistory(
    messages=[
        HumanMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
        AIMessage(content='I would like to know about Langchain.', additional_kwargs={}, response_metadata={})
    ]
)

In [7]:
# Retrieve and print the conversation history
for message in history.messages:
	print(f"{message.type}: {message.content}")

human: How can I assist you today?
ai: I would like to know about Langchain.


In [1]:
from operator import itemgetter
from typing import List

from langchain_openai.chat_models import ChatOpenAI

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


history = get_by_session_id("1")
history.add_message(AIMessage(content="hello"))
print(store)  # noqa: T201

{'1': InMemoryHistory(messages=[AIMessage(content='hello', additional_kwargs={}, response_metadata={})])}


In [10]:
from typing import Optional
from typing import List
from langchain_community.chat_models import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.chat_history import BaseChatMessageHistory
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage, AIMessage


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

store = {}

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])
llm = ChatOllama(model="llama3.2:1b")
chain = prompt | llm 

chain_with_history = RunnableWithMessageHistory(
    chain,
    # Uses the get_by_session_id function defined in the example
    # above.
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

pp(chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "foo"}}
))

# Uses the store defined in the example above.
pp(store)  # noqa: T201

pp(chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"session_id": "foo"}}
))

pp(store)  # noqa: T201

AIMessage(
    content='In mathematics, the term "cosine" refers to a fundamental trigonometric function that describes the 
ratio of the length of the adjacent side to the angle to the length of the hypotenuse in a right-angled 
triangle.\n\nThe cosine of an angle is defined as the ratio of the adjacent side (the side next to the angle) to 
the hypotenuse (the side opposite the angle). It\'s denoted by the Greek letter "cos" and is usually calculated 
using the formula:\n\ncos(θ) = opposite side / hypotenuse\n\nFor example, if we have a right-angled triangle with 
an angle θ, the cosine of that angle would be the ratio of the length of the adjacent side (the side next to the 
angle) to the length of the hypotenuse. It\'s a very useful function in many areas of mathematics and science, 
including geometry, trigonometry, physics, and engineering.\n\nIt\'s worth noting that there are also other 
trigonometric functions like sine (sin), tangent (tan), and cotangent (cot), which have different definitions and 
formulas. But cosine is one of the most well-known and widely used trigonometric functions!',
    additional_kwargs={},
    response_metadata={
        'model': 'llama3.2:1b',
        'created_at': '2025-01-13T21:16:13.164532Z',
        'message': {'role': 'assistant', 'content': ''},
        'done_reason': 'stop',
        'done': True,
        'total_duration': 1964195333,
        'load_duration': 31882666,
        'prompt_eval_count': 39,
        'prompt_eval_duration': 284000000,
        'eval_count': 239,
        'eval_duration': 1647000000
    },
    id='run-236a5cc2-6807-48f2-ad3b-ea5859d365ff-0',
    usage_metadata={'input_tokens': 39, 'output_tokens': 239, 'total_tokens': 278}
)

{
    'foo': InMemoryHistory(
        messages=[
            HumanMessage(content='What does cosine mean?', additional_kwargs={}, response_metadata={}),
            AIMessage(
                content='In mathematics, the term "cosine" refers to a fundamental trigonometric function that 
describes the ratio of the length of the adjacent side to the angle to the length of the hypotenuse in a 
right-angled triangle.\n\nThe cosine of an angle is defined as the ratio of the adjacent side (the side next to the
angle) to the hypotenuse (the side opposite the angle). It\'s denoted by the Greek letter "cos" and is usually 
calculated using the formula:\n\ncos(θ) = opposite side / hypotenuse\n\nFor example, if we have a right-angled 
triangle with an angle θ, the cosine of that angle would be the ratio of the length of the adjacent side (the side 
next to the angle) to the length of the hypotenuse. It\'s a very useful function in many areas of mathematics and 
science, including geometry, trigonometry, physics, and engineering.\n\nIt\'s worth noting that there are also 
other trigonometric functions like sine (sin), tangent (tan), and cotangent (cot), which have different definitions
and formulas. But cosine is one of the most well-known and widely used trigonometric functions!',
                additional_kwargs={},
                response_metadata={
                    'model': 'llama3.2:1b',
                    'created_at': '2025-01-13T21:16:13.164532Z',
                    'message': {'role': 'assistant', 'content': ''},
                    'done_reason': 'stop',
                    'done': True,
                    'total_duration': 1964195333,
                    'load_duration': 31882666,
                    'prompt_eval_count': 39,
                    'prompt_eval_duration': 284000000,
                    'eval_count': 239,
                    'eval_duration': 1647000000
                },
                id='run-236a5cc2-6807-48f2-ad3b-ea5859d365ff-0',
                usage_metadata={'input_tokens': 39, 'output_tokens': 239, 'total_tokens': 278}
            )
        ]
    )
}

AIMessage(
    content='The inverse of the cosine function is the sine function. In other words, if you square both sides of 
the equation cos(θ) = opposite side / hypotenuse, you can solve for θ.\n\nThe formula to find the angle θ from the 
cosine value is:\n\nsin(θ) = √(1 - cos²(θ))\n\nThis formula is derived from the Pythagorean identity, which states 
that in a right-angled triangle, the square of the length of the hypotenuse (the side opposite the angle) is equal 
to the sum of the squares of the lengths of the other two sides.\n\nFor example, if you know the cosine value of an
angle θ, you can plug it into this formula to find the sine value.',
    additional_kwargs={},
    response_metadata={
        'model': 'llama3.2:1b',
        'created_at': '2025-01-13T21:16:14.170844Z',
        'message': {'role': 'assistant', 'content': ''},
        'done_reason': 'stop',
        'done': True,
        'total_duration': 992990417,
        'load_duration': 10890708,
        'prompt_eval_count': 291,
        'prompt_eval_duration': 16000000,
        'eval_count': 151,
        'eval_duration': 963000000
    },
    id='run-44a8b36e-8670-4244-9835-75f0191cb033-0',
    usage_metadata={'input_tokens': 291, 'output_tokens': 151, 'total_tokens': 442}
)

{
    'foo': InMemoryHistory(
        messages=[
            HumanMessage(content='What does cosine mean?', additional_kwargs={}, response_metadata={}),
            AIMessage(
                content='In mathematics, the term "cosine" refers to a fundamental trigonometric function that 
describes the ratio of the length of the adjacent side to the angle to the length of the hypotenuse in a 
right-angled triangle.\n\nThe cosine of an angle is defined as the ratio of the adjacent side (the side next to the
angle) to the hypotenuse (the side opposite the angle). It\'s denoted by the Greek letter "cos" and is usually 
calculated using the formula:\n\ncos(θ) = opposite side / hypotenuse\n\nFor example, if we have a right-angled 
triangle with an angle θ, the cosine of that angle would be the ratio of the length of the adjacent side (the side 
next to the angle) to the length of the hypotenuse. It\'s a very useful function in many areas of mathematics and 
science, including geometry, trigonometry, physics, and engineering.\n\nIt\'s worth noting that there are also 
other trigonometric functions like sine (sin), tangent (tan), and cotangent (cot), which have different definitions
and formulas. But cosine is one of the most well-known and widely used trigonometric functions!',
                additional_kwargs={},
                response_metadata={
                    'model': 'llama3.2:1b',
                    'created_at': '2025-01-13T21:16:13.164532Z',
                    'message': {'role': 'assistant', 'content': ''},
                    'done_reason': 'stop',
                    'done': True,
                    'total_duration': 1964195333,
                    'load_duration': 31882666,
                    'prompt_eval_count': 39,
                    'prompt_eval_duration': 284000000,
                    'eval_count': 239,
                    'eval_duration': 1647000000
                },
                id='run-236a5cc2-6807-48f2-ad3b-ea5859d365ff-0',
                usage_metadata={'input_tokens': 39, 'output_tokens': 239, 'total_tokens': 278}
            ),
            HumanMessage(content="What's its inverse", additional_kwargs={}, response_metadata={}),
            AIMessage(
                content='The inverse of the cosine function is the sine function. In other words, if you square 
both sides of the equation cos(θ) = opposite side / hypotenuse, you can solve for θ.\n\nThe formula to find the 
angle θ from the cosine value is:\n\nsin(θ) = √(1 - cos²(θ))\n\nThis formula is derived from the Pythagorean 
identity, which states that in a right-angled triangle, the square of the length of the hypotenuse (the side 
opposite the angle) is equal to the sum of the squares of the lengths of the other two sides.\n\nFor example, if 
you know the cosine value of an angle θ, you can plug it into this formula to find the sine value.',
                additional_kwargs={},
                response_metadata={
                    'model': 'llama3.2:1b',
                    'created_at': '2025-01-13T21:16:14.170844Z',
                    'message': {'role': 'assistant', 'content': ''},
                    'done_reason': 'stop',
                    'done': True,
                    'total_duration': 992990417,
                    'load_duration': 10890708,
                    'prompt_eval_count': 291,
                    'prompt_eval_duration': 16000000,
                    'eval_count': 151,
                    'eval_duration': 963000000
                },
                id='run-44a8b36e-8670-4244-9835-75f0191cb033-0',
                usage_metadata={'input_tokens': 291, 'output_tokens': 151, 'total_tokens': 442}
            )
        ]
    )
}